This program takes the information from the Observations, Objects, and iraf_dr5.1 tables and combines it into a single table, with a row per object. 

It also pulls information for each target from APASS, 2MASS, and UCAC4 based on RA/Dec matching. 

In [2]:
import pandas as pd
import os
import numpy as np
from astropy.io import fits
from astropy.time import Time

Read in the table of observations produced by Janez.

In [3]:
obs_data=pd.read_csv("/Users/kschles/Documents/GALAH/iraf_v5.1/observations_05042016.csv", low_memory=False)

Isolate science observations. Here I remove calibration images (biases/flats/arcs). I also extract all observations with CCD1 because I don't need to know all the information from each CCD. 

In [4]:
obs_data['ccd']=((obs_data['runccd_id'].astype(str)).str[10:11]).astype(int)
obs1_data=obs_data.loc[np.where((obs_data['ccd']==1) & ((obs_data['ndfclass_updated']=='MFOBJECT') | (obs_data['ndfclass_updated']=='MFFLX')))[0]]

Group the observations by cob_id. 

In [5]:
obs1_data.columns.values

array(['runccd_id', 'run_id', 'dirname', 'fitsfile', 'run', 'cob_id',
       'ccd', 'plate', 'cfg_file', 'meanra', 'meandec', 'cenra', 'cendec',
       'utmjd', 'ndfclass', 'ndfclass_updated', 'fieldid', 'obsid',
       'exposed', 'std_name', 'obstatus', 'qflag', 'oclass'], dtype=object)

In [6]:
temp=obs1_data.groupby(['cob_id'], as_index=False)
cobid_obsdata=temp.agg({'runccd_id': 'first', 'dirname': 'first', 'ccd' : 'first', 'plate' : 'first', 'cfg_file': 'first', 'cenra': 'first', 'cendec': 'first', 'obsid': 'first', 'exposed': np.sum, 'std_name': 'first', 'qflag': 'first', 'oclass': 'first','ndfclass_updated': 'first', 'utmjd' : np.mean})

Set up UTMJD formatting so it can be modified to other formats. 

In [7]:
t=Time(cobid_obsdata['utmjd'], format='mjd', scale='utc')

In [9]:
print t.iso[0]

2013-11-10 12:08:53.997


Read in the IRAF output table 

In [8]:
iraf_data=pd.read_csv("/Users/kschles/Documents/GALAH/iraf_v5.1/iraf_dr51_new_05042016.csv")
iraf_data.rename(columns=lambda x: x.strip(), inplace=True)

In [13]:
iraf_data.columns.values

array(['sobject_id', 'galah_id', 'cob_id', 'pivot', 'field_id', 'ra',
       'dec', 'ebv', 'barycentric', 'rv_c1_guess', 'rv_c2_guess',
       'rv_c3_guess', 'rv_c4_guess', 'rv_guess', 'e_rv_guess',
       'teff_guess', 'logg_guess', 'feh_guess', 'flag_guess',
       'snr_c1_guess', 'snr_c2_guess', 'snr_c3_guess', 'snr_c4_guess',
       'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf',
       'snr2_c1_iraf', 'snr2_c2_iraf', 'snr2_c3_iraf', 'snr2_c4_iraf',
       'red_flag', 'red_vers'], dtype=object)

In [9]:
## The GUESS values are often set to 999. or 9999. rather than null. So I need to adjust this. 
## Note also that v_ccd4 is invalid; it is merely a copy of v_comb. 
iraf_data.loc[np.where(iraf_data['rv_c2_guess'].astype(float)==999.)[0],'rv_c2_guess']=np.nan

Now we want to match the IRAF output table with the observations table based on cob_id. Each object will now have information about the field observation in its row. 

In [10]:
combo1=pd.merge(iraf_data,cobid_obsdata[['cenra', 'cendec', 'qflag', 'std_name', 'cfg_file', 'obsid', 'ndfclass_updated', 'cob_id', 'runccd_id', 'dirname']], how="left", on="cob_id")

I now want to match up the information in combo1 (observations+IRAF output) with the information from the objects table because I want info like objects.name and objects.comment. The objects table is huge, so, using Jeffrey's bash script, I've split it up by night. We match up the big_combo table with the objects table on a night by night basis using cob_id and pivot. 

In [10]:
combo1.columns.values

array(['sobject_id', 'galah_id', 'cob_id', 'pivot', 'field_id', 'ra',
       'dec', 'ebv', 'barycentric', 'rv_c1_guess', 'rv_c2_guess',
       'rv_c3_guess', 'rv_c4_guess', 'rv_guess', 'e_rv_guess',
       'teff_guess', 'logg_guess', 'feh_guess', 'flag_guess',
       'snr_c1_guess', 'snr_c2_guess', 'snr_c3_guess', 'snr_c4_guess',
       'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf',
       'snr2_c1_iraf', 'snr2_c2_iraf', 'snr2_c3_iraf', 'snr2_c4_iraf',
       'red_flag', 'red_vers', 'cenra', 'cendec', 'qflag', 'std_name',
       'cfg_file', 'obsid', 'ndfclass_updated', 'runccd_id', 'dirname'], dtype=object)

In [11]:
## This groups the objects in combo1 table by date and creates a list of each date used. 
date_grouping=combo1.groupby('dirname')
date_list=combo1[['dirname']].groupby('dirname', as_index=False).first()

In [12]:
for i in range(len(date_list)):
    ## Use the aggregate table to pull out each individual date
    date_name=np.array(date_list.loc[i])[0].astype(int)
    print date_name
    ## Pulls out all of the target observations for that night, organised by COB_ID and Pivot
    extract=date_grouping.get_group(date_name)
    
    ## Read in the objects table for that night
    filename='/Users/kschles/Documents/GALAH/iraf_v5.1/objects_by_date/'+date_name.astype(str)+'.txt'
    temp_objects=pd.read_csv(filename, names=['runccd_id','run_id','pivot','fibre','type','ra','dec','x','y','xerr','yerr','theta','object_name','comment','mag','pmra','pmdec','pid','retractor','wlen','galah_id','out_name','airmass','barycentric','heliocentric'],index_col=False, low_memory=False)
    
    ### REDO HOW THIS MATCHES THINGS UP???
    y=pd.merge(extract, temp_objects[['runccd_id', 'pivot', 'object_name', 'comment', 'mag', 'galah_id']], how="left", on=['runccd_id', 'pivot'])
    
    ## the result dataframe has the combination of big_combo with the objects table and we append the combo from each night. 
    if (i==0) : 
        result=y
    else :
        result=result.append(y)


131116
131118
131119
131120
131121
131122
131123
131216
131218
131220
131222
140112
140114
140117
140118
140207
140208
140209
140210
140211
140212
140303
140304
140305
140307
140308
140309
140310
140311
140312
140313
140314
140315
140316
140409
140410
140412
140413
140414
140415
140607
140608
140609
140610
140611
140707
140708
140709
140710
140711
140712
140713
140805
140806
140807
140809
140813
140814
141101
141102
141103
141104
141202
141231
150101
150102
150103
150105
150106
150107
150108
150109
150112
150204
150205
150206
150207
150208
150209
150210
150211
150330
150401
150403
150404
150405
150406
150407
150408
150409
150410
150411
150412
150413
150426
150427
150428
150429
150430
150504
150531
150601
150602
150603
150604
150605
150607
150703
150704
150705
150706
150718
150719
150824
150826
150827
150828
150829
150830
150831
150901
150902
150903
151008
151009
151109
151110
151111
151219
151220
151222
151224
151225
151226
151228
151229
151230
151231
160106
160107
160108
160109
160110

The result table has all objects, even those observed as MFFLX stars. Thus, it needs to be cleaned up. 

In [13]:
result.reset_index(drop=True, inplace=True)

In [16]:
## Clean up the table columns.
result2=result.loc[np.where(result['ndfclass_updated'].str.match('MFOBJECT', as_indexer=True)==True)[0]]
result2.reset_index(drop=True, inplace=True)
result2.rename(columns={'galah_id_x' : 'galah_id'}, inplace=True)


Use STILTS to match up the table with 2MASS, APASS, and UCAC4. Searching for a match for each target in the result2 table within 1 arcsecond. 

2MASS catalog is II/246/out 

UCAC4 catalog is 'I/322A/out' 

PPMXL is 'I/317'

SPM is 'I/320'

The USNO-B catalog is 'I/284/out'

If the OS command returns a 0, all is well with the matching. 

In [18]:
## First output the file to a temporary csv. 
result2[['sobject_id', 'ra', 'dec']].to_csv('result_temp.csv', index=False)

In [19]:
## Then query 2MASS with these targets. stilts.jar must be in your working directory. 
os.system('java -jar stilts.jar cdsskymatch cdstable=II/246/out find=each in=result_temp.csv ifmt=csv ra=ra dec=dec radius=1 out=result_temp_2mass.csv')

0

In [20]:
## Query UCAC4 with STILTS and target list 
os.system('java -jar stilts.jar cdsskymatch cdstable=I/322A/out find=each in=result_temp.csv ifmt=csv ra=ra dec=dec radius=1 out=result_temp_ucac4.csv')

0

In [21]:
## Query APASS with STILTS and target list 
os.system('java -jar stilts.jar cdsskymatch cdstable=II/336/apass9 find=each in=result_temp.csv ifmt=csv ra=ra dec=dec radius=1 out=result_temp_apass.csv')

0

Once you have the relevant data tables, match them with the results2 table based on cob_id and pivot. 

In [22]:
## Read in the data you just pulled. 
twomass_data=pd.read_csv('result_temp_2mass.csv')
ucac4_data=pd.read_csv('result_temp_ucac4.csv')
#usnob_data=pd.read_csv('result_temp_usnob.csv')
apass_data=pd.read_csv('result_temp_apass.csv')

In [23]:
temp1=pd.merge(result2, twomass_data[['sobject_id', '2MASS', 'Jmag', 'e_Jmag', \
                                     'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag', 'Qfl', 'Rfl', 'X']], how='left', on=['sobject_id'])
temp2=pd.merge(temp1, ucac4_data[['sobject_id', 'UCAC4', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE']], how='left', on=['sobject_id'])
combined_table=pd.merge(temp2, apass_data[['sobject_id', 'Vmag', 'e_Vmag', 'Bmag', 'e_Bmag', 
                                 'gpmag', 'e_gpmag', 'rpmag', 'e_rpmag', 'ipmag', 'e_ipmag']], how='left', on=['sobject_id'])

In [24]:
## Adjust column names
combined_table.rename(columns={'2MASS' : '2MASS_ID', 'UCAC4' : 'UCAC4_ID', 'teff' : 'TEFF_GUESS', \
                              'logg' : 'LOGG_GUESS', 'feh' : 'FEH_GUESS', 'qflag' : 'RED_QFLAG', \
                              'galah_id' : 'GALAH_ID'}, inplace=True)
## Shift all to upper case
combined_table.columns = [x.upper() for x in combined_table.columns]

Now I output the resulting combined table as a FITS file to use for other programs. 

In [25]:
combined_table.loc[np.where(combined_table['GALAH_ID'].isnull()==True)[0], 'GALAH_ID']=-1

In [28]:
combined_table.rename(columns=lambda x: x.lower(), inplace=True)
combined_table.columns.values

array(['sobject_id', 'galah_id', 'cob_id', 'pivot', 'field_id', 'ra',
       'dec', 'ebv', 'barycentric', 'rv_c1_guess', 'rv_c2_guess',
       'rv_c3_guess', 'rv_c4_guess', 'rv_guess', 'e_rv_guess',
       'teff_guess', 'logg_guess', 'feh_guess', 'flag_guess',
       'snr_c1_guess', 'snr_c2_guess', 'snr_c3_guess', 'snr_c4_guess',
       'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf',
       'snr2_c1_iraf', 'snr2_c2_iraf', 'snr2_c3_iraf', 'snr2_c4_iraf',
       'red_flag', 'red_vers', 'cenra', 'cendec', 'red_qflag', 'std_name',
       'cfg_file', 'obsid', 'ndfclass_updated', 'runccd_id', 'dirname',
       'object_name', 'comment', 'mag', 'galah_id_y', '2mass_id', 'jmag',
       'e_jmag', 'hmag', 'e_hmag', 'kmag', 'e_kmag', 'qfl', 'rfl', 'x',
       'ucac4_id', 'pmra', 'e_pmra', 'pmde', 'e_pmde', 'vmag', 'e_vmag',
       'bmag', 'e_bmag', 'gpmag', 'e_gpmag', 'rpmag', 'e_rpmag', 'ipmag',
       'e_ipmag'], dtype=object)

In [29]:
combined_table['placeholder']=None

In [47]:
## Output to FITS file 

## Target Information
col01=fits.Column(name='sobject_id', format='K15', array=combined_table['sobject_id'])
col02=fits.Column(name='date',format='I6', array=combined_table['dirname'])
col03=fits.Column(name='galah_id',format='K15',null=-1, array=combined_table['galah_id'])
col04=fits.Column(name='ra',format='F',unit='deg', array=combined_table['ra'])
col05=fits.Column(name='dec',format='F',unit='deg', array=combined_table['dec'])
col06=fits.Column(name='field_id',format='A50',unit='', array=combined_table['cfg_file'])
col07=fits.Column(name='program_id',format='A20',unit='', array=combined_table['placeholder'])
col08=fits.Column(name='utmjd',format='E',unit='', array=combined_table['placeholder'])
col09=fits.Column(name='eff_midtexp',format='E', unit='s', array=combined_table['placeholder'])
col10=fits.Column(name='eff_texp',format='E', unit='s', array=combined_table['placeholder'])
col11=fits.Column(name='object_name',format='A20', array=combined_table['object_name'])
col12=fits.Column(name='comment',format='A20', array=combined_table['comment'])
col13=fits.Column(name='barycentric',format='F', array=combined_table['barycentric'])
col14=fits.Column(name='ebv',format='F',unit='mag', array=combined_table['ebv'])
col15=fits.Column(name='red_vers',format='F',unit='', array=combined_table['red_vers'])
col16=fits.Column(name='red_flag',format='F',unit='', array=combined_table['red_flag'])
col17=fits.Column(name='snr_c1_iraf', format='F', unit='', array=combined_table['snr_c1_iraf'])
col18=fits.Column(name='snr_c2_iraf', format='F', unit='', array=combined_table['snr_c2_iraf'])
col19=fits.Column(name='snr_c3_iraf', format='F', unit='', array=combined_table['snr_c3_iraf'])
col20=fits.Column(name='snr_c4_iraf', format='F', unit='', array=combined_table['snr_c4_iraf'])
col21=fits.Column(name='snr_c1_guess', format='F', unit='', array=combined_table['snr_c1_guess'])
col22=fits.Column(name='snr_c2_guess', format='F', unit='', array=combined_table['snr_c2_guess'])
col23=fits.Column(name='snr_c3_guess', format='F', unit='', array=combined_table['snr_c3_guess'])
col24=fits.Column(name='snr_c4_guess', format='F', unit='', array=combined_table['snr_c4_guess'])
## 2MASS Information 
col25=fits.Column(name='id_tmass',format='A20', unit='',array=combined_table['2mass_id'])
col26=fits.Column(name='j_tmass',format='E', unit='mag',array=combined_table['jmag'])
col27=fits.Column(name='e_j_tmass',format='E', unit='mag',array=combined_table['e_jmag'])
col28=fits.Column(name='h_tmass',format='E', unit='mag',array=combined_table['hmag'])
col29=fits.Column(name='e_h_tmass',format='E', unit='mag',array=combined_table['e_hmag'])
col30=fits.Column(name='k_tmass',format='E', unit='mag',array=combined_table['kmag'])
col31=fits.Column(name='e_k_tmass',format='E', unit='mag',array=combined_table['e_kmag'])
col32=fits.Column(name='qfl_tmass',format='A5', unit='',array=combined_table['qfl'])
col33=fits.Column(name='rfl_tmass',format='I3', unit='',array=combined_table['rfl'])
col34=fits.Column(name='xfl_tmass',format='I1', unit='',array=combined_table['x'])
## APASS Information 
col35=fits.Column(name='b_apass',format='E', unit='mag',array=combined_table['bmag'])
col36=fits.Column(name='e_b_apass',format='E', unit='mag',array=combined_table['e_bmag'])
col37=fits.Column(name='v_apass',format='E', unit='mag',array=combined_table['vmag'])
col38=fits.Column(name='e_v_apass',format='E', unit='mag',array=combined_table['e_vmag'])
col39=fits.Column(name='g_apass',format='E', unit='mag',array=combined_table['gpmag'])
col40=fits.Column(name='e_g_apass',format='E', unit='mag',array=combined_table['e_gpmag'])
col41=fits.Column(name='r_apass',format='E', unit='mag',array=combined_table['rpmag'])
col42=fits.Column(name='e_r_apass',format='E', unit='mag',array=combined_table['e_rpmag'])
col43=fits.Column(name='i_apass',format='E', unit='mag',array=combined_table['ipmag'])
col44=fits.Column(name='e_i_apass',format='E', unit='mag',array=combined_table['e_ipmag'])
## UCAC4 Information 
col45=fits.Column(name='id_ucac4',format='A10', unit='',array=combined_table['ucac4_id'])
col46=fits.Column(name='pmra_ucac4',format='E', unit='mas/yr',array=combined_table['pmra'])
col47=fits.Column(name='e_pmra_ucac4',format='E', unit='mas/yr',array=combined_table['e_pmra'])
col48=fits.Column(name='pmdec_ucac4',format='E', unit='mas/yr',array=combined_table['pmde'])
col49=fits.Column(name='e_pmdec_ucac4',format='E', unit='mas/yr',array=combined_table['e_pmde'])



In [48]:
#get_ipython().system(u'rm iraf_output_combination.fits')
cols=fits.ColDefs([col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,\
                   col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,\
                   col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,\
                   col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,\
                   col41,col42,col43,col44,col45,col46,col47,col48,col49])
tbhdu=fits.BinTableHDU.from_columns(cols)
tbhdu.writeto('sobject_iraf_general.fits')


output to csv with comparable information??

IN THE FUTURE WE CAN COMBINE WITH ADDITIONAL INFORMATION, LIKE VALUES FROM TOMAZ AND JANE